<a href="https://colab.research.google.com/github/y22ecprojectb3-max/speech/blob/main/wav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Path to the main zip file
main_zip_path = '/content/drive/MyDrive/DS_10283_1942.zip'

# Final dataset directory
dataset_dir = '/content/dataset'
os.makedirs(dataset_dir, exist_ok=True)

# Open the main zip file
with zipfile.ZipFile(main_zip_path, 'r') as main_zip_ref:
    # Loop through all files inside the main zip file
    for file_info in main_zip_ref.infolist():
        file_name = file_info.filename

        # Only process .zip files within the main zip
        if file_name.endswith(".zip"):
            # Create a target directory for each inner zip file
            target_dir = os.path.join(dataset_dir, os.path.splitext(file_name)[0])
            os.makedirs(target_dir, exist_ok=True)

            # Extract the inner zip file
            main_zip_ref.extract(file_info, dataset_dir)

            # Extract the contents of the inner zip file into its target directory
            inner_zip_path = os.path.join(dataset_dir, file_name)
            with zipfile.ZipFile(inner_zip_path, 'r') as inner_zip_ref:
                inner_zip_ref.extractall(target_dir)
                print(f"✅ Extracted {file_name} → {target_dir}")

            # Remove the extracted inner zip file after extraction
            os.remove(inner_zip_path)

In [ ]:
# Show first 10 files in the noisy training set
!ls /content/dataset/noisy_trainset_wav | head -n 10


In [ ]:
import os

# Check extracted folders
for root, dirs, files in os.walk('/content/dataset'):
    print(f"\nFolder: {root}")
    for f in files[:10]:  # show only first 10 files
        print("   ", f)

In [ ]:
!pip install speechbrain pydub

import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement
from pydub import AudioSegment
import matplotlib.pyplot as plt
import IPython.display as ipd

In [ ]:
# Load SpeechBrain's speech enhancement model
enhancer = SpectralMaskEnhancement.from_hparams(
    source="speechbrain/metricgan-plus-voicebank",
    savedir="pretrained_models/metricgan-plus-voicebank"
)


In [ ]:
import os

# Check clean/noisy speech files
print("Clean trainset samples:", len(os.listdir("/content/dataset/clean_trainset_wav")))
print("Noisy trainset samples:", len(os.listdir("/content/dataset/noisy_trainset_wav")))
print("Clean testset samples:", len(os.listdir("/content/dataset/clean_testset_wav")))
print("Noisy testset samples:", len(os.listdir("/content/dataset/noisy_testset_wav")))

# Show a few transcript files
print("\nTranscript samples:")
print(os.listdir("/content/dataset/trainset_txt")[:10])


In [ ]:
import IPython.display as ipd
import random
import scipy.io.wavfile as wavfile

# Pick a random noisy and clean file
noisy_file = "/content/dataset/noisy_trainset_wav/p226_001.wav"
clean_file = "/content/dataset/clean_trainset_wav/p226_001.wav"

# Read audio files
rate_noisy, audio_noisy = wavfile.read(noisy_file)
rate_clean, audio_clean = wavfile.read(clean_file)

print("Noisy Speech:")
ipd.display(ipd.Audio(audio_noisy, rate=rate_noisy))

print("Clean Speech (Ground Truth):")
ipd.display(ipd.Audio(audio_clean, rate=rate_clean))

In [ ]:
import IPython.display as ipd
import scipy.io.wavfile as wavfile
import os

# Example: play noisy and clean speech
noisy_file = "/content/dataset/noisy_trainset_wav/p226_001.wav"
clean_file = "/content/dataset/clean_trainset_wav/p226_001.wav"
enhanced_file = "enhanced_speech.wav" # replace with your output file

# Read audio files
rate_noisy, audio_noisy = wavfile.read(noisy_file)
rate_clean, audio_clean = wavfile.read(clean_file)

print("🔊 Noisy Speech:")
ipd.display(ipd.Audio(audio_noisy, rate=rate_noisy))

print("🔊 Clean Speech (Reference):")
ipd.display(ipd.Audio(audio_clean, rate=rate_clean))

# Check if enhanced file exists before trying to display it
if os.path.exists(enhanced_file):
    rate_enhanced, audio_enhanced = wavfile.read(enhanced_file)
    print("🔊 Enhanced Speech (after model):")
    ipd.display(ipd.Audio(audio_enhanced, rate=rate_enhanced))
else:
    print(f"Enhanced speech file not found at {enhanced_file}. Please run your model to generate it.")

In [ ]:
import IPython.display as ipd
import scipy.io.wavfile as wavfile
import os

# Example: play noisy and clean speech
noisy_file = "/content/dataset/noisy_trainset_wav/p226_001.wav"
clean_file = "/content/dataset/clean_trainset_wav/p226_001.wav"
enhanced_file = "enhanced_speech.wav" # replace with your output file

# Read audio files
rate_noisy, audio_noisy = wavfile.read(noisy_file)
rate_clean, audio_clean = wavfile.read(clean_file)

print("🔊 Noisy Speech:")
ipd.display(ipd.Audio(audio_noisy, rate=rate_noisy))

print("🔊 Clean Speech (Reference):")
ipd.display(ipd.Audio(audio_clean, rate=rate_clean))

# Check if enhanced file exists before trying to display it
if os.path.exists(enhanced_file):
    rate_enhanced, audio_enhanced = wavfile.read(enhanced_file)
    print("🔊 Enhanced Speech (after model):")
    ipd.display(ipd.Audio(audio_enhanced, rate=rate_enhanced))
else:
    print(f"Enhanced speech file not found at {enhanced_file}. Please run your model to generate it.")

In [ ]:
!ls /content/dataset/noisy_trainset_wav | head -n 10


In [ ]:
!pip install speechbrain
import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement


In [ ]:
enhancer = SpectralMaskEnhancement.from_hparams(
    source="speechbrain/metricgan-plus-voicebank",
    savedir="pretrained_models/metricgan-plus-voicebank"
)


In [ ]:
noisy_file = "/content/dataset/noisy_trainset_wav/p226_010.wav"  # change to a real file
waveform, fs = torchaudio.load(noisy_file)

print("Noisy Speech:")
import IPython.display as ipd
ipd.display(ipd.Audio(noisy_file))


In [ ]:
import torch

In [ ]:
import torch
import torchaudio
import IPython.display as ipd

enhanced = enhancer.enhance_batch(waveform, lengths=torch.tensor([1.0]))

# Save enhanced speech
torchaudio.save("enhanced_speech.wav", enhanced.cpu(), fs)

print("✅ Enhanced speech saved as enhanced_speech.wav")
ipd.display(ipd.Audio("enhanced_speech.wav"))

In [ ]:
import torch
import torchaudio
import IPython.display as ipd

enhanced = enhancer.enhance_batch(waveform, lengths=torch.tensor([1.0]))

# Save enhanced speech
torchaudio.save("enhanced_speech.wav", enhanced.cpu(), fs)

print("✅ Enhanced speech saved as enhanced_speech.wav")
ipd.display(ipd.Audio("enhanced_speech.wav"))

In [ ]:
import matplotlib.pyplot as plt

enhanced_waveform, fs = torchaudio.load("enhanced_speech.wav")

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(waveform[0].numpy())
plt.title("Noisy Speech")

plt.subplot(1, 2, 2)
plt.plot(enhanced_waveform[0].numpy())
plt.title("Enhanced Speech")

plt.show()


In [ ]:
import IPython.display as ipd

# Example: play noisy and clean speech
noisy_file = "/content/dataset/noisy_trainset_wav/p226_010.wav"
clean_file = "/content/dataset/clean_trainset_wav/p226_010.wav"

print("🔊 Noisy Speech:")
ipd.display(ipd.Audio(noisy_file))

print("🔊 Clean Speech (Reference):")
ipd.display(ipd.Audio(clean_file))

print("🔊 Enhanced Speech (after model):")
ipd.display(ipd.Audio("enhanced_speech.wav"))  # replace with your output file


In [ ]:
!ls /content/dataset/noisy_trainset_wav | head -n 10


In [ ]:
import torchaudio
import matplotlib.pyplot as plt

waveform, sr = torchaudio.load(noisy_file)
enhanced_waveform, sr = torchaudio.load("enhanced_speech.wav")  # your output

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(waveform[0].numpy())
plt.title("Noisy Speech")

plt.subplot(1, 2, 2)
plt.plot(enhanced_waveform[0].numpy())
plt.title("Enhanced Speech")

plt.show()


In [ ]:
import IPython.display as ipd

# Example: play noisy and clean speech
noisy_file = "/content/dataset/noisy_trainset_wav/p226_009.wav"
clean_file = "/content/dataset/clean_trainset_wav/p226_009.wav"

print("🔊 Noisy Speech:")
ipd.display(ipd.Audio(noisy_file))

print("🔊 Clean Speech (Reference):")
ipd.display(ipd.Audio(clean_file))

print("🔊 Enhanced Speech (after model):")
ipd.display(ipd.Audio("enhanced_speech.wav"))  # replace with your output file


In [ ]:
!ls /content/dataset/noisy_trainset_wav | head -n 10


In [ ]:
import torchaudio
import matplotlib.pyplot as plt

waveform, sr = torchaudio.load(noisy_file)
enhanced_waveform, sr = torchaudio.load("enhanced_speech.wav")  # your output

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(waveform[0].numpy())
plt.title("Noisy Speech")

plt.subplot(1, 2, 2)
plt.plot(enhanced_waveform[0].numpy())
plt.title("Enhanced Speech")

plt.show()
